In [1]:
import pandas as pd
import os
import glob
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
#info from https://data.binance.vision/?prefix=data/spot/monthly/klines/BTCUSDT/5m/

C:\Users\Jordi\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "input_ada/*.csv"))

In [3]:
df_all = pd.DataFrame()
for f in csv_files:
    df = pd.read_csv(f, names=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
    df_all = pd.concat([df,df_all])

In [4]:
df_all

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1651363200000,0.7555,0.7598,0.7542,0.7597,1288872.2,1651363499999,9.753360e+05,1816,593963.6,449613.96411,0
1,1651363500000,0.7597,0.7613,0.7577,0.7612,1083449.6,1651363799999,8.232017e+05,1521,505000.2,383726.35391,0
2,1651363800000,0.7612,0.7650,0.7582,0.7617,1845440.3,1651364099999,1.406306e+06,2824,1050025.7,799924.41876,0
3,1651364100000,0.7617,0.7626,0.7603,0.7609,989416.0,1651364399999,7.535015e+05,1324,520225.0,396157.67509,0
4,1651364400000,0.7608,0.7615,0.7587,0.7597,955595.0,1651364699999,7.258976e+05,1474,512468.0,389239.04238,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8923,1643672100000,1.0520,1.0550,1.0520,1.0520,242239.9,1643672399999,2.551595e+05,490,117976.9,124277.05080,0
8924,1643672400000,1.0530,1.0530,1.0490,1.0490,369557.4,1643672699999,3.884496e+05,523,146845.4,154397.87860,0
8925,1643672700000,1.0500,1.0510,1.0490,1.0500,174839.3,1643672999999,1.835454e+05,344,99919.3,104916.77400,0
8926,1643673000000,1.0500,1.0520,1.0490,1.0520,345385.7,1643673299999,3.629725e+05,373,239851.3,252097.90170,0


In [5]:
df_all = df_all.sort_values('Open time', ascending=True)

In [6]:
df_all['date'] = pd.to_datetime(df_all['Open time'], unit='ms', origin='unix')

In [7]:
df_all = df_all.set_index('date')

In [8]:
df_all = df_all[df_all.index > '2022-01-01']

In [9]:
df_all['RSI_14'] = df_all.ta.rsi(length=14)

In [10]:
df_all['Signal_long'] = np.where((df_all['RSI_14'] > 50) & (df_all.shift(1)['RSI_14'] < 50), 1 ,0)

In [11]:
df_all['Signal_short'] = np.where((df_all['RSI_14'] < 50) & (df_all.shift(1)['RSI_14'] > 50), 2 ,0)

In [12]:
df_all['Signal'] = df_all[['Signal_long', 'Signal_short']].max(axis=1)

In [13]:
df_all['Signal_long_close'] = np.where((df_all['Signal_long'] > 0).shift(11).fillna(False), 3, 0)

In [14]:
df_all['Signal_short_close'] = np.where((df_all['Signal_short'] > 0).shift(11).fillna(False), 4, 0)

In [15]:
def get_signal():
    return df_all[14:].Signal

In [16]:
def get_signal_longc():
    return df_all[14:].Signal_long_close

In [17]:
def get_signal_shortc():
    return df_all[14:].Signal_short_close

In [18]:
from backtesting import Strategy
class strat2(Strategy):
    def init(self):
        #super().init()
        self.signal1 = self.I(get_signal)
        self.signal2 = self.I(get_signal_longc)
        self.signal3 = self.I(get_signal_shortc)
        self.counter = 0
        
    def next(self):
        #super().next()
        self.counter = self.counter + 1
        #print(self.counter, self.trades)
        if self.signal1==1:
            #print(self.counter, self.trades)
            #sl1 = self.ha_min[-1]
            #tp1 = self.data.ha_close[-1] + 600e-4
            self.buy(size=.10) #sl=sl1) #, tp=tp1)
        if self.signal1==2:
            #print(self.counter, self.trades)
            #sl1 = self.ha_high[-1]
            #tp1 = self.data.Close[-1] - 600e-4
            self.sell(size=.10) #sl=sl1) #, tp=tp1)
        if self.signal2 == 3:
            for i in self.trades[:]:
                if i.is_long:
                    i.close()
                    break
        if self.signal3 == 4:
            for i in self.trades[:]:
                if i.is_short:
                    i.close()
                    break

In [20]:
bt_l = Backtest(df_all[14:], strat2, cash=100_000, commission=.002, hedging=True)  # ,exclusive_orders=True)
stat_l = bt_l.run()
stat_l

Start                     2022-01-01 01:15:00
End                       2022-05-31 23:55:00
Duration                    150 days 22:40:00
Exposure Time [%]                     64.5688
Equity Final [$]                      36107.1
Equity Peak [$]                        100000
Return [%]                           -63.8929
Buy & Hold Return [%]                -53.0838
Return (Ann.) [%]                    -91.3439
Volatility (Ann.) [%]                 0.65019
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -63.9133
Avg. Drawdown [%]                    -63.9133
Max. Drawdown Duration      150 days 21:20:00
Avg. Drawdown Duration      150 days 21:20:00
# Trades                                 5564
Win Rate [%]                          39.2164
Best Trade [%]                        7.65088
Worst Trade [%]                      -9.59346
Avg. Trade [%]                    

In [21]:
stat_l._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7556,17,28,1.323348,1.3270,-27.594512,-0.002760,2022-01-01 02:40:00,2022-01-01 03:35:00,0 days 00:55:00
1,6753,18,29,1.331658,1.3270,-31.455474,-0.003498,2022-01-01 02:45:00,2022-01-01 03:40:00,0 days 00:55:00
2,-6108,19,30,1.325344,1.3280,-16.222848,-0.002004,2022-01-01 02:50:00,2022-01-01 03:45:00,0 days 00:55:00
3,5469,25,36,1.331658,1.3160,-85.633602,-0.011758,2022-01-01 03:20:00,2022-01-01 04:15:00,0 days 00:55:00
4,-4949,26,37,1.324346,1.3160,41.304354,0.006302,2022-01-01 03:25:00,2022-01-01 04:20:00,0 days 00:55:00
...,...,...,...,...,...,...,...,...,...,...
5559,4844,43440,43451,0.604707,0.6115,32.905292,0.011234,2022-05-31 21:15:00,2022-05-31 22:10:00,0 days 00:55:00
5560,-4385,43441,43452,0.600397,0.6087,-36.409532,-0.013830,2022-05-31 21:20:00,2022-05-31 22:15:00,0 days 00:55:00
5561,3896,43442,43453,0.606010,0.6102,16.325798,0.006915,2022-05-31 21:25:00,2022-05-31 22:20:00,0 days 00:55:00
5562,-3534,43445,43456,0.602093,0.6130,-38.543924,-0.018114,2022-05-31 21:40:00,2022-05-31 22:35:00,0 days 00:55:00


In [22]:
stat_l._trades.query('Size > 0').PnL.sum()

-29493.861124400028

In [23]:
stat_l._trades.query('Size < 0').PnL.sum()

-34399.008127200075

In [24]:
stat_l._trades.to_csv('strat2test.csv')